In [15]:
from glob import glob
import os.path as osp
import os

import numpy as np
import pandas as pd

val_ratio=0.2

data_root="data"
img_dir="flashAmbient"
img_subdirs=["People_Photos","Objects_Photos",
         "Plants_Photos","Rooms_Photos",
         "Shelves_Photos","Toys_Photos"]
mask_dir="SynShadow/matte"


img_names=np.random.permutation(np.array([osp.join(subdir,img_name) for subdir in img_subdirs 
    for img_name in sorted(os.listdir(
        osp.join(data_root,img_dir,subdir)))]).reshape((-1,2)))
mask_file_list=np.random.permutation(
            np.array(os.listdir(osp.join(data_root,mask_dir))))

val_img_names,train_img_names=np.split(img_names, [int(val_ratio*len(img_names))])
val_mask_list, train_mask_list=np.split(mask_file_list,[int(val_ratio*len(mask_file_list))])

train_img_df=pd.DataFrame(train_img_names,columns=["ambient","flash"])
val_img_df=pd.DataFrame(val_img_names,columns=["ambient","flash"])

train_mask_df=pd.DataFrame(train_mask_list,columns=["mask"])
val_mask_df=pd.DataFrame(val_mask_list,columns=["mask"])

train_img_df.to_csv(osp.join(data_root,img_dir,"train_list.csv"),index=False)
val_img_df.to_csv(osp.join(data_root,img_dir,"val_list.csv"),index=False)

train_mask_df.to_csv(osp.join(data_root,mask_dir,"train_list.csv"),index=False)
val_mask_df.to_csv(osp.join(data_root,mask_dir,"val_list.csv"),index=False)



ImportError: cannot import name 'save_png' from 'utils.data' (/data/xjiangan/flash-reflection-removal/utils/data.py)

In [6]:
from __future__ import division
from glob import glob
import os.path as osp
import os

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from utils.data import load_img, save_png , darken, gen_shadow2

data_root="data"
mask_dir=osp.join(data_root,"SynShadow/matte")
img_dir=osp.join(data_root,"flashAmbient")
img_subdirs=["People_Photos","Objects_Photos",
         "Plants_Photos","Rooms_Photos",
         "Shelves_Photos","Toys_Photos"]

output_dir=osp.join(data_root,"SythISTD")

train_out=osp.join(output_dir,"train")
test_out=osp.join(output_dir,"test")
for p in(train_out,test_out):
    if not osp.exists(p):
        os.makedirs(p)

train_mask_list=pd.read_csv(osp.join(mask_dir,"train_list.csv"))["mask"].map(
    lambda x:osp.join(mask_dir,x)).tolist()
val_mask_list =pd.read_csv(osp.join(mask_dir,"val_list.csv"))["mask"].map(
    lambda x:osp.join(mask_dir,x)).tolist()


def gen_shadows(img_dir,mask_dir,list_file,output_dir,out_prefix):
    mask_list=pd.read_csv(osp.join(mask_dir,list_file))["mask"].map(
        lambda x:osp.join(mask_dir,x)).tolist()
    img_names=pd.read_csv(osp.join(img_dir,list_file))["ambient"].map(
        lambda x:osp.join(img_dir,x)).tolist()

    mask_file_list=tf.constant(mask_list)
    n=len(img_names)
    img_names=np.array(img_names)
    num_img=img_names.shape[0]

    ds=tf.data.Dataset.from_tensor_slices(img_names)
    ds=ds.map(lambda x:gen_shadow2(x,mask_file_list),num_parallel_calls=4).prefetch(2)
    classes=["_A","_B","_C"]

    for j in range(5):
        iterator=ds.make_one_shot_iterator()
        next_imgs=iterator.get_next()
        with tf.Session() as sess:
            for i in range(n):
                imgs=sess.run(next_imgs)
                for k, cls in enumerate(classes):
                    sess.run(tf.io.write_file(
                        osp.join(output_dir,out_prefix+cls,"%04d-%01d.png"%(i,j)),
                        imgs[k]))
    return imgs



In [7]:
imgs=gen_shadows(img_dir,mask_dir,"val_list.csv",test_out,"test")

KeyboardInterrupt: 